In [ ]:
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
import os
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from tqdm import tqdm
sns.set_palette('Dark2')
sns.set_context('paper')
sns.set_style({'axes.axisbelow': True, 
               'axes.edgecolor': '.15',
               'axes.facecolor': 'white',
               'axes.grid': True, 
               'axes.labelcolor': '.15', 
               'figure.facecolor': 'white', 
               'grid.color': '.15',
               'grid.linestyle': ':', 
               'grid.alpha': .5, 
               'image.cmap': 'Greys', 
               'legend.frameon': False, 
               'legend.numpoints': 1, 
               'legend.scatterpoints': 1,
               'lines.solid_capstyle': 'butt', 
               'axes.spines.right': False, 
               'axes.spines.top': False,  
               'text.color': '.15',  
               'xtick.top': False, 
               'ytick.right': False, 
               'xtick.color': '.15',
               'xtick.direction': 'out', 
               'ytick.color': '.15', 
               'ytick.direction': 'out', 
              })


import matplotlib

FONT_SIZE_PT = 5
matplotlib.rcParams['font.family'] = 'Arial'
matplotlib.rcParams['font.size'] = FONT_SIZE_PT
matplotlib.rcParams['axes.labelsize'] = FONT_SIZE_PT
matplotlib.rcParams['axes.titlesize'] = FONT_SIZE_PT
matplotlib.rcParams['figure.titlesize'] = FONT_SIZE_PT
matplotlib.rcParams['xtick.labelsize'] = FONT_SIZE_PT
matplotlib.rcParams['ytick.labelsize'] = FONT_SIZE_PT
matplotlib.rcParams['legend.fontsize'] = FONT_SIZE_PT
matplotlib.rcParams['legend.title_fontsize'] = FONT_SIZE_PT

matplotlib.rcParams['xtick.major.size'] = matplotlib.rcParams['ytick.major.size'] = 2
matplotlib.rcParams['xtick.major.width'] = matplotlib.rcParams['ytick.major.width'] = 0.5


matplotlib.rcParams['xtick.minor.size'] = matplotlib.rcParams['ytick.minor.size'] = 1

matplotlib.rcParams['xtick.minor.width'] = matplotlib.rcParams['ytick.minor.width'] = 0.5

matplotlib.rcParams['axes.linewidth'] = 0.5
matplotlib.rcParams['lines.linewidth'] = 0.5
matplotlib.rcParams['grid.linewidth'] = 0.25
matplotlib.rcParams['patch.linewidth'] = 0.25
matplotlib.rcParams['lines.markeredgewidth'] = 0.25
matplotlib.rcParams['lines.markersize'] = 2

FIVE_MM_IN_INCH = 0.19685
DPI = 600
matplotlib.rcParams['figure.figsize'] = (10 * FIVE_MM_IN_INCH, 9 * FIVE_MM_IN_INCH)
matplotlib.rcParams['savefig.dpi'] = DPI
matplotlib.rcParams['figure.dpi'] = DPI // 4


#http://phyletica.org/matplotlib-fonts/
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

# (05) Excel output

## Configuration

In [ ]:
import pathlib

INPUT_MARCS_DATA = pathlib.Path('outputs') / '02-linking-to-MARCS' / '03-mapping-between-chip-ms-and-marcs-features.csv'
assert INPUT_MARCS_DATA.is_file()

INPUT_GENE_LABEL_MAP = pathlib.Path('outputs') / '02-linking-to-MARCS' / '02-marcs-to-chip-ms-mapping.csv'
assert INPUT_GENE_LABEL_MAP.is_file()

INPUT_MODEL_RESULTS = pathlib.Path('outputs') / '03-transformation-and-modelling' / '07-output-full_results.csv'
assert INPUT_MODEL_RESULTS.is_file()

INPUT_CHIP_MS_METADATA = pathlib.Path('outputs') / '01-extracting' / 'data_metadata.csv'
assert INPUT_CHIP_MS_METADATA.is_file()

INPUT_NUMERIC_DATA = pathlib.Path('outputs') / '01-extracting' / 'data_numeric.csv'
assert INPUT_NUMERIC_DATA.is_file()

In [ ]:
MODEL_COEFS = [
    'H3K4me1vsControl',
    'H3K4me3vsControl',
    'H3K4me3vsH3K4me1',
]

MARCS_FEATURE_ORDER = [
    # Same order as in Fig 3
    'H2A.Z', 'meDNA', 
    'H3K4me1', 'H3K4me3', 'H3ac', 'H3K9acK14ac', 'H3K27ac', 
    'H3K9me2', 'H3K9me3', 'H3K27me2', 'H3K27me3',
    'H4ac', 'H4K16ac', 'H4K20me2', 'H4K20me3',         
]

In [ ]:
import pathlib
OUTPUT_DIRECTORY = pathlib.Path('outputs') / '05-excel-output'

if not OUTPUT_DIRECTORY.is_dir():
    OUTPUT_DIRECTORY.mkdir(parents=True)

## Reading

### MARCS

We only need the prepared feature data from previous notebooks

In [ ]:
data_marcs_features = pd.read_csv(INPUT_MARCS_DATA, index_col=0)
data_marcs_features.columns = pd.MultiIndex.from_tuples([c.split('__') for c in data_marcs_features.columns], names=['marcs_stat', 'marcs_feature'])
data_marcs_features

### Metadata

In [ ]:
data_metadata_chip_ms = pd.read_csv(INPUT_CHIP_MS_METADATA, index_col=0)
data_metadata_chip_ms

## Gene label map

In [ ]:
data_gene_label_map = pd.read_csv(INPUT_GENE_LABEL_MAP, index_col=0)
data_gene_label_map

Make the ChIP-MS <-> MARCS gene label map centred on the ChIP-MS labels, by concatenating multiple mappings via ';'

In [ ]:
data_gene_label_map_gene_centric = pd.DataFrame({
    'marcs_gene_label': data_gene_label_map.groupby('chip_ms_label')['marcs_gene_label'].apply(lambda x: ';'.join(sorted(x.unique()))),
    # Split the already ';' separated 'mapped_via' column, and then re-join it after deduplicating
    'mapped_via': data_gene_label_map.groupby('chip_ms_label')['mapped_via'].apply(lambda x: ';'.join(sorted(x.str.split(';', expand=True).stack().unique())))
})
data_gene_label_map_gene_centric

In [ ]:
data_gene_label_map_gene_centric['mapped_via'].value_counts()

### Unnormalised data

In [ ]:
data_unnormalised_numeric = pd.read_csv(
    INPUT_NUMERIC_DATA,
    index_col=0
)
data_unnormalised_numeric.columns.name = 'Experiment_Replicate'
data_unnormalised_numeric

### Model outputs

Let's read the model outputs:

In [ ]:
data_model_outputs = pd.read_csv(INPUT_MODEL_RESULTS, index_col=0)
data_model_outputs.columns = pd.MultiIndex.from_tuples([c.split('__') for c in data_model_outputs.columns])
data_model_outputs

In [ ]:
full_model_data_columns = data_model_outputs.columns.get_level_values(0).unique()
full_model_data_columns

And split the normalised data away from the coefficients, away from other data

In [ ]:
data_model_outputs_normalised_data = data_model_outputs['normalised_data']
data_model_outputs_coefficients = data_model_outputs['coefficient_estimates']
data_model_outputs_estimates = data_model_outputs[full_model_data_columns.difference(['normalised_data', 'coefficient_estimates'])]

In [ ]:
data_model_outputs_normalised_data

In [ ]:
data_model_outputs_coefficients

In [ ]:
data_model_outputs_estimates

## Collecting the data for output

### ChIP-MS Data

It always makes sense to start with metadata

In [ ]:
excel_metadata = data_metadata_chip_ms.copy()

# Copy the label into metadata so we don't nee to reset index
excel_metadata['Label'] = excel_metadata.index

# also add MARCS gene label info
excel_metadata = excel_metadata.join(data_gene_label_map_gene_centric)

# Some columns should be before others
_first_columns = ['Label', 'Accession', 'Gene', 'marcs_gene_label', 'mapped_via', 'Description'] 
excel_metadata = excel_metadata[_first_columns + list(excel_metadata.columns.difference(_first_columns))]

# Rename marcs gene label column
excel_metadata = excel_metadata.rename(columns={"marcs_gene_label": 'MARCS Gene label(s)', 'mapped_via': "MARCS Gene label(s) linked via"})

# Add a header columns
excel_metadata.columns = pd.MultiIndex.from_tuples([('metadata', c) for c in excel_metadata.columns])

excel_metadata

#### Raw Data

Now let's shape the raw data into something more useful too

In [ ]:
excel_raw_data = data_unnormalised_numeric.copy()
# Add a header column
excel_raw_data.columns = pd.MultiIndex.from_tuples([('raw_data', c) for c in excel_raw_data.columns])

excel_raw_data

#### Normalised Data

Likewise for normalised data

In [ ]:
excel_normalised_data = data_model_outputs_normalised_data.copy()

# Add a header column
excel_normalised_data.columns = pd.MultiIndex.from_tuples([('normalised_data_log2', c) for c in excel_normalised_data.columns])

excel_normalised_data

### MARCS Data

From MARCS data we really only need the MARCS feature classifications.

In [ ]:
data_marcs_features[[
    'significant_category_strong',
    'significant_category_weak'
]]

For excel we will recode these into shorter counterparts:

- R/E - Recruited/Excluded
- SR/SE - Strongly Recruited/Strongly Excluded
- N - Neither
- (empty) - No Data


In [ ]:
def categorise_features_for_excel(row):
    if row[['significant_category_weak', 'significant_category_strong']].isnull().any():
        # One being null should imply both null
        assert row[['significant_category_weak', 'significant_category_strong']].isnull().all()    
        return None
    elif row['significant_category_strong'] != 'Neither':
        return {'Strongly recruited': 'SR', 'Strongly excluded': 'SE'}[row['significant_category_strong']]
    else:
        return {'Recruited': 'R', 'Excluded': 'E', 'Neither': 'N'}[row['significant_category_weak']]
    

_df = data_marcs_features[[
    'significant_category_strong',
    'significant_category_weak'
]].swaplevel(axis='columns')

excel_marcs_features = {}
for marcs_feature in MARCS_FEATURE_ORDER:
    excel_marcs_features[marcs_feature] = _df[marcs_feature].apply(categorise_features_for_excel, axis=1)
    
excel_marcs_features = pd.DataFrame(excel_marcs_features)

# Add two headers as well, why not
excel_marcs_features.columns = pd.MultiIndex.from_tuples([('marcs_feature_effects', c) for c in excel_marcs_features.columns])

### Model outputs

Finally, the model outputs. We will split the outputs as follows:

1. We will take the logFC_imputed columns and place them separtely (as a summary)
2. We will also separate the comment
3. And the coefficient estimates
4. We will keep remaining outputs pretty much as is (separately)

In [ ]:
full_model_output_estimates_columns = data_model_outputs_estimates.columns.get_level_values(1).unique()
full_model_output_estimates_columns

In [ ]:
excel_model_comment = data_model_outputs_estimates[[('comment', 'comment')]].copy()
excel_model_comment.columns = pd.MultiIndex.from_tuples([('model_estimates', 'comment')])
excel_model_comment

In [ ]:
excel_model_coefficients = data_model_outputs_coefficients.copy()
excel_model_coefficients.columns = pd.MultiIndex.from_tuples([('model_coefficient_estimates', c) for c in excel_model_coefficients.columns])
excel_model_coefficients

In [ ]:
excel_model_outputs_fc_estimates = data_model_outputs_estimates.loc(axis=1)[MODEL_COEFS, 'logFC_imputed']
excel_model_outputs_fc_estimates.columns = pd.MultiIndex.from_tuples([('model_estimates_for_{}'.format(c[0]), c[1]) for c in excel_model_outputs_fc_estimates.columns])
excel_model_outputs_fc_estimates

In [ ]:
excel_model_outputs_other = data_model_outputs_estimates[MODEL_COEFS]
excel_model_outputs_other = excel_model_outputs_other[[c for c in excel_model_outputs_other.columns if c[1] != 'logFC_imputed']].copy()
excel_model_outputs_other.columns = pd.MultiIndex.from_tuples([('model_estimates_for_{}'.format(c[0]), c[1]) for c in excel_model_outputs_other.columns])
excel_model_outputs_other

## Writing excel

In [ ]:
import xlsxwriter
from seaborn.utils import relative_luminance

Collect everything in one place

In [ ]:
df_excel = excel_metadata.join(excel_model_outputs_fc_estimates).join(excel_model_comment).join(excel_marcs_features).join(excel_model_outputs_other).join(excel_model_coefficients).join(excel_normalised_data).join(excel_raw_data)
assert not df_excel.index.duplicated().any()
df_excel

Sort by me3vsme1 difference, followed by the other two coefs, descending

In [ ]:
df_excel = df_excel.sort_values(by=[
    ('model_estimates_for_H3K4me3vsH3K4me1', 'logFC_imputed'),
    ('model_estimates_for_H3K4me3vsControl', 'logFC_imputed'),
    ('model_estimates_for_H3K4me1vsControl', 'logFC_imputed'),
], ascending=False)

In [ ]:
output_xlsx = OUTPUT_DIRECTORY / '01-model-results.xlsx'
output_csv = OUTPUT_DIRECTORY / '01-model-results.csv.gz'

COLUMN_GROUPS = {
    'Metadata': excel_metadata.columns,
    'Log2(FC) estimates (incl. imputed)': list(excel_model_outputs_fc_estimates.columns),
    'Comment': list(excel_model_comment.columns),
    'MARCS Feature Response (SR/SE: Strongly recruited/excluded, R/E: Recruited/Excluded, N: neither)': list(excel_marcs_features.columns),
}

GROUP_DEPTHS = {
    # 'Comment': 2, # Merging the two rows for the comment group makes excel file not sortable
}

for coef in MODEL_COEFS:
    COLUMN_GROUPS[f'Model outputs for {coef}'] = list(excel_model_outputs_other[[f'model_estimates_for_{coef}']].columns)

COLUMN_GROUPS = {
    **COLUMN_GROUPS,
    'Model coefficient estimates': list(excel_model_coefficients),
    'Normalised data (log2)': list(excel_normalised_data),
    'Raw data': list(excel_raw_data),
}

RENAMES = {}

for col in excel_model_outputs_fc_estimates.columns:
    RENAMES[col] = col[0].partition('model_estimates_for_')[2]
    
_simple_renames = []
_simple_renames.extend(excel_metadata.columns)
_simple_renames.extend(excel_marcs_features.columns)
_simple_renames.extend(excel_model_outputs_other.columns)
_simple_renames.extend(excel_model_coefficients.columns)
_simple_renames.extend(excel_normalised_data.columns)
_simple_renames.extend(excel_raw_data.columns)
_simple_renames.extend(excel_model_comment.columns)

for col in _simple_renames:
    RENAMES[col] = col[1]
    

writer = pd.ExcelWriter(output_xlsx, engine='xlsxwriter')
workbook = writer.book

bold = workbook.add_format({'bold': True})
bold_right = workbook.add_format({'bold': True, 'right': 1})

bold_rotated = workbook.add_format({'bold': True, 'rotation':90})
bold_rotated_right = workbook.add_format({'bold': True, 'rotation':90, 'right': 1})

merged_format = workbook.add_format({
    'bold': 1,
    'align': 'center',
    'valign': 'vcenter',
    'right': 1,
    'text_wrap': True,
})

right_border = workbook.add_format({
    'right': 1,
})

sheet_name = f"Summary"

first_data_row = 2
first_data_col = 0

# pandas does not support writing to Excel with multi-index and header=False
# so quickly remove the index, from the DF we're writing
# (it won't make a difference as we will handle header columns ourselves)
_df_excel_nomultiindex = df_excel.copy()
_df_excel_nomultiindex.columns = ['__'.join(map(str, c)) for c in df_excel.columns]
# Write this df as csv:
_df_excel_nomultiindex.to_csv(output_csv)
# And into excel
_df_excel_nomultiindex.to_excel(
    writer, 
    sheet_name=sheet_name, 
    startrow=first_data_row, 
    startcol=first_data_col, 
    index=False, 
    header=False
)

last_data_row = first_data_row + len(df_excel)
last_data_col = first_data_col + len(df_excel.columns)

worksheet = writer.sheets[sheet_name]

SEPARATOR_COLUMNS = {v[-1] for v in COLUMN_GROUPS.values()}

colname_to_index_map = {}
for i, col in enumerate(df_excel.columns, start=first_data_col):
    fmt_ = bold_rotated if not col in SEPARATOR_COLUMNS else bold_rotated_right
    
    worksheet.write(first_data_row-1, i, RENAMES.get(col, str(col)), fmt_)
    colname_to_index_map[col] = i

for merged_name, col_list in COLUMN_GROUPS.items():
    _first = colname_to_index_map[col_list[0]]
    _last = colname_to_index_map[col_list[-1]]
    
    rows_to_merge = GROUP_DEPTHS.get(merged_name, 1)
    
    if _first == _last and rows_to_merge == 1:
        # Cannot merge one column only
        worksheet.write(first_data_row-2, _first, merged_name, merged_format)
    else:
        worksheet.merge_range(
            first_data_row-2, colname_to_index_map[col_list[0]], 
            first_data_row-2+(rows_to_merge-1), colname_to_index_map[col_list[-1]],
            merged_name,
            merged_format
        )
        
for col in SEPARATOR_COLUMNS:
    worksheet.set_column(colname_to_index_map[col], colname_to_index_map[col], cell_format=right_border)

color_red = '#b2182b'
format_red = workbook.add_format({'bg_color': color_red})
color_red_lighter = '#f4a582'

color_white = '#f7f7f7'

color_blue = '#2166ac'
format_blue = workbook.add_format({'bg_color': color_blue})
color_blue_lighter = '#92c5de'

for (val, color) in [('SR', color_red), ('SE', color_blue), ('R', color_red_lighter), ('E', color_blue_lighter)]:
    for col in COLUMN_GROUPS['MARCS Feature Response (SR/SE: Strongly recruited/excluded, R/E: Recruited/Excluded, N: neither)']:
        fmt_ = workbook.add_format({
            'bg_color': color,
            'font_color': "#000000" if relative_luminance(color) > .408 else "#FFFFFF"
        })

        worksheet.conditional_format(
            first_data_row, colname_to_index_map[col], 
            last_data_row, colname_to_index_map[col],
            {
                'type': 'cell',
                'criteria': 'equal to',
                'value': f'"{val}"',
                'format': fmt_,
            }
        )
       
    
three_color_columns = list(COLUMN_GROUPS['Log2(FC) estimates (incl. imputed)'])
three_color_columns += [c for c in excel_model_outputs_other.columns if c[1] == 'logFC']


for col in three_color_columns:
    
    _limit = df_excel[col].abs().replace(np.inf, np.nan).quantile(0.98)
    print(f"Excel formatting limit for {col}: +/-{_limit}")
    
    worksheet.conditional_format(
        first_data_row, colname_to_index_map[col], 
        last_data_row, colname_to_index_map[col],
        {
            'type': '3_color_scale',
            'min_type': 'num',
            'max_type': 'num',
            'mid_type': 'num',
            'min_value': -_limit,
            'max_value': _limit,
            'mid_value': 0,
            'max_color': color_red,
            'mid_color': color_white,
            'min_color': color_blue,

        }
    )
    
    worksheet.conditional_format(
        first_data_row, colname_to_index_map[col], 
        last_data_row, colname_to_index_map[col], 
        {'type': 'cell',
        'criteria': '==',
        'value': '"inf"',
        'format': format_red}
    )
    
    worksheet.conditional_format(
        first_data_row, colname_to_index_map[col], 
        last_data_row, colname_to_index_map[col], 
        {'type': 'cell',
        'criteria': '==',
        'value': '"-inf"',
        'format': format_blue}
    )
    
databar_columns = list(COLUMN_GROUPS['Model coefficient estimates']) + list(excel_normalised_data.columns) + list(excel_raw_data.columns) 

for col in databar_columns:
    
    worksheet.conditional_format(
        first_data_row, colname_to_index_map[col], 
        last_data_row, colname_to_index_map[col],
        {
            'type': 'data_bar',
            'min_type': 'percentile',
            'max_type': 'percentile',
            'min_value': 1,
            'max_value': 99,

        }
    )
    

boolean_columns_good = [c for c in excel_model_outputs_other.columns if c[1] in ['significant']]
boolean_columns_bad = [c for c in excel_model_outputs_other.columns if c[1] in ['logFC_is_imputed', 'logFC_based_on_single_datapoint']]

bold_green = workbook.add_format({'bold': True, 'font_color': '#1b7837'})
bold_red = workbook.add_format({'bold': True, 'font_color': color_red})
for col in boolean_columns_good:
    worksheet.conditional_format(
        first_data_row, colname_to_index_map[col], 
        last_data_row, colname_to_index_map[col], 
        {'type': 'cell',
        'criteria': '==',
        'value': 'TRUE',
        'format': bold_green}
    )

for col in boolean_columns_bad:
    worksheet.conditional_format(
        first_data_row, colname_to_index_map[col], 
        last_data_row, colname_to_index_map[col], 
        {'type': 'cell',
        'criteria': '==',
        'value': 'TRUE',
        'format': bold_red}
    )
    
# Narrow columns
narrow_columns = list(excel_marcs_features.columns)
narrow_columns.extend([c for c in data_metadata_chip_ms if c[1] in ['# PSMs', '# Peptides', '# Unique Peptides', 'Coverage [%]']])
    
for col in narrow_columns:
    worksheet.set_column(colname_to_index_map[col], colname_to_index_map[col], 4)

worksheet.set_column(colname_to_index_map[('metadata', 'Description')], colname_to_index_map[('metadata', 'Description')], 40)
worksheet.set_column(colname_to_index_map[('model_estimates', 'comment')], colname_to_index_map[('model_estimates', 'comment')], 40)

worksheet.freeze_panes(first_data_row, colname_to_index_map[('metadata', 'MARCS Gene label(s)')] + 1)
worksheet.autofilter(first_data_row-1, first_data_col, last_data_row, last_data_col)
      
writer.save()
print("Done!")

In [ ]:
excel_metadata